In [1]:
from data_preparation import load_data_tensor, split_train_data

In [2]:
import torch
import numpy as np
import torch.optim as optim
from sklearn.model_selection import KFold
from GSR_Net.preprocessing import *
from GSR_Net.model import *
from GSR_Net.train import *
import argparse

seed_value = 42
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [3]:
# Check for CUDA availability
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print(f"CUDA available... Using {DEVICE}!")
else:
    DEVICE = torch.device('cpu')
    print(f"CUDA not available... Using {DEVICE}!")

CUDA available... Using cuda!


In [4]:
path_to_data = '/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/data'

x_train, x_test, y_train = load_data_tensor(path_to_data)

In [5]:
X, Y = x_train.to(DEVICE), y_train.to(DEVICE)

In [6]:
class ModelArgs:  
    def __init__(self, epochs, lr, splits, lmbda, lr_dim, hr_dim, hidden_dim,
                 padding):
        self.epochs = epochs
        self.lr = lr
        self.splits = splits
        self.lmbda = lmbda
        self.lr_dim = lr_dim
        self.hr_dim = hr_dim
        self.hidden_dim = hidden_dim
        self.padding = padding

args = ModelArgs(
    epochs=200,
    lr=0.0001,
    splits=3,
    lmbda=16,
    lr_dim=160,
    #hr_dim=320,
    #hidden_dim=320,
    hr_dim=320,
    hidden_dim=320,
    padding=26,
)

In [7]:
cv = KFold(n_splits=args.splits, random_state=42, shuffle=True)
ks = [0.9, 0.7, 0.6, 0.5]


In [8]:
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args).to(DEVICE)
print(model)

GSRNet(
  (layer): GSRLayer()
  (net): GraphUnet(
    (start_gcn): GCN(
      (proj): Linear(in_features=160, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (bottom_gcn): GCN(
      (proj): Linear(in_features=320, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (end_gcn): GCN(
      (proj): Linear(in_features=640, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
  )
  (gc1): GraphConvolution()
  (gc2): GraphConvolution()
)


/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_Net/layers.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.weights = torch.nn.Parameter(data=torch.tensor(initial_weights, device=self.device, dtype=torch.float), requires_grad=True)


In [10]:
Y.size()

torch.Size([167, 268, 268])

In [9]:
def train_model():
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    for train_index, test_index in cv.split(X):
        subjects_adj, test_adj, subjects_ground_truth, test_ground_truth = X[
            train_index], X[test_index], Y[train_index], Y[test_index]
        train(model, optimizer, subjects_adj, subjects_ground_truth, args)
        test(model, test_adj, test_ground_truth, args)
    

### L1 Loss

In [10]:
train_model()

/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_Net/preprocessing.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float32)


Epoch:  1 Loss:  0.8746659424071922 Error:  25.988830446748807 %
Epoch:  2 Loss:  0.5940021142027432 Error:  25.91338502733331 %
Epoch:  3 Loss:  0.45791336602734445 Error:  25.802313004221233 %
Epoch:  4 Loss:  0.3801361371700029 Error:  25.585741936264185 %
Epoch:  5 Loss:  0.33066079580694213 Error:  24.85881592322113 %
Epoch:  6 Loss:  0.297476619706118 Error:  23.79708613892247 %
Epoch:  7 Loss:  0.28054673443163247 Error:  23.24816090495963 %
Epoch:  8 Loss:  0.27178493411021126 Error:  22.929271092092186 %
Epoch:  9 Loss:  0.2674908011703563 Error:  22.7088814615307 %
Epoch:  10 Loss:  0.2655556626561889 Error:  22.53211713150928 %
Epoch:  11 Loss:  0.264357515519723 Error:  22.397799957963755 %
Epoch:  12 Loss:  0.2632356515728441 Error:  22.28932780654807 %
Epoch:  13 Loss:  0.26203043243490665 Error:  22.203688973322848 %
Epoch:  14 Loss:  0.2612084239945376 Error:  22.132100276929094 %
Epoch:  15 Loss:  0.26038244411461337 Error:  22.069637466194038 %
Epoch:  16 Loss:  0.259

### MSE Loss - 5 fold

In [10]:
train_model()

/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_Net/preprocessing.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float32)


Epoch:  1 Loss:  0.15706203872323932 Error:  11.826408503198982 %
Epoch:  2 Loss:  0.1340201700988569 Error:  11.67988578291764 %
Epoch:  3 Loss:  0.1231388312981541 Error:  11.212810523864022 %
Epoch:  4 Loss:  0.10775670173921083 Error:  9.844356750075082 %
Epoch:  5 Loss:  0.09632093159663946 Error:  8.892203889843216 %
Epoch:  6 Loss:  0.09155151844584852 Error:  8.545486651417008 %
Epoch:  7 Loss:  0.08909848060710986 Error:  8.374220813463506 %
Epoch:  8 Loss:  0.08753207152275215 Error:  8.260571472066685 %
Epoch:  9 Loss:  0.08629966177102319 Error:  8.163081297188773 %
Epoch:  10 Loss:  0.08521716069794238 Error:  8.073975498739042 %
Epoch:  11 Loss:  0.08435332352169474 Error:  8.002121906195368 %
Epoch:  12 Loss:  0.08361585023707914 Error:  7.939359811799866 %
Epoch:  13 Loss:  0.08302055758641179 Error:  7.888731646134441 %
Epoch:  14 Loss:  0.08252030300130521 Error:  7.844967420275946 %
Epoch:  15 Loss:  0.08209658744639921 Error:  7.806821377541786 %
Epoch:  16 Loss:  0

### MSE Loss - 3 fold

In [13]:
train_model()

/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_Net/preprocessing.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float32)


Epoch:  1 Loss:  0.16108746853497652 Error:  12.011011146210334 %
Epoch:  2 Loss:  0.13893885111755078 Error:  11.909015731768566 %
Epoch:  3 Loss:  0.12947023693505708 Error:  11.681652834286561 %
Epoch:  4 Loss:  0.11965846901272868 Error:  10.964772430760366 %
Epoch:  5 Loss:  0.10500314677352304 Error:  9.586329435979998 %
Epoch:  6 Loss:  0.09550765210443789 Error:  8.79048372845392 %
Epoch:  7 Loss:  0.09081630962523254 Error:  8.431443763343063 %
Epoch:  8 Loss:  0.08805209576143874 Error:  8.22589228953327 %
Epoch:  9 Loss:  0.08622335085460732 Error:  8.089026963120109 %
Epoch:  10 Loss:  0.0848438814618029 Error:  7.981831875738797 %
Epoch:  11 Loss:  0.08369886663717192 Error:  7.889186849330996 %
Epoch:  12 Loss:  0.08273997352466927 Error:  7.8095988110379055 %
Epoch:  13 Loss:  0.08182308883280368 Error:  7.728060324852532 %
Epoch:  14 Loss:  0.08096182953801241 Error:  7.651936289694933 %
Epoch:  15 Loss:  0.0802676105485843 Error:  7.592082825732661 %
Epoch:  16 Loss:  

In [15]:
pred = model(x_test[0])[0]

In [16]:
pred.size()

torch.Size([320, 320])

### changing dimensions

In [10]:
train_model()

/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_Net/preprocessing.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float32)


Epoch:  1 Loss:  0.16181752978412955 Error:  12.016799858024529 %
Epoch:  2 Loss:  0.13899467737825066 Error:  11.910859314171043 %
Epoch:  3 Loss:  0.12939201067159842 Error:  11.669347015721304 %
Epoch:  4 Loss:  0.1191648701960976 Error:  10.906066722026816 %
Epoch:  5 Loss:  0.10520414967794676 Error:  9.62503775283023 %
Epoch:  6 Loss:  0.09697163390280965 Error:  8.965907756004247 %
Epoch:  7 Loss:  0.0929867048991156 Error:  8.67351026416899 %
Epoch:  8 Loss:  0.09063703751376083 Error:  8.505678193660469 %
Epoch:  9 Loss:  0.08899638764895834 Error:  8.384562055538366 %
Epoch:  10 Loss:  0.0877037895423872 Error:  8.28312394116913 %
Epoch:  11 Loss:  0.08667402781613238 Error:  8.200932169953983 %
Epoch:  12 Loss:  0.08577500075638832 Error:  8.127078826765755 %
Epoch:  13 Loss:  0.08506195606277869 Error:  8.068160702650612 %
Epoch:  14 Loss:  0.08442874856897302 Error:  8.015793041737231 %
Epoch:  15 Loss:  0.08391292463686015 Error:  7.971075371847497 %
Epoch:  16 Loss:  0.0

In [11]:
torch.save(model.state_dict(), '/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/GSR_models/model.pth')

In [17]:
x_test = x_test.to(DEVICE)

In [18]:
x_test.size()

torch.Size([112, 160, 160])

In [43]:
from data_preparation import *

In [60]:
def find_preds_and_convert_to_submission(file_name, model=model, test_set=x_test):
    preds_list = []
    with torch.no_grad():
        for test_data in test_set:
            pred, _, _, _ = model(test_data)
            pred = unpad(pred, args.padding)
            preds_list.append(pred)
    pred_tensor = torch.stack(preds_list).cpu().numpy()
    generate_submission_file(pred_tensor, f'/vol/bitbucket/km2120/DGL_Project/DGL24-Group-Project/submission_files/{file_name}')        

In [61]:
find_preds_and_convert_to_submission('GSR-Net_1')

In [53]:
pred_tensor = torch.stack(preds_list)
pred_tensor.size()

torch.Size([112, 268, 268])

In [49]:
pred_tensor

array([[0.16849859, 0.17459072, 0.5030451 , ..., 0.27178514, 0.11517152,
        0.13508317],
       [0.30288285, 0.3185417 , 0.52418387, ..., 0.24824302, 0.09390321,
        0.10113391],
       [0.21680379, 0.23106878, 0.28092152, ..., 0.11293958, 0.01763152,
        0.10326452],
       ...,
       [0.3341934 , 0.48751003, 0.48240575, ..., 0.18306269, 0.06877226,
        0.12171288],
       [0.1753784 , 0.15432212, 0.3039307 , ..., 0.14235291, 0.05198485,
        0.11434773],
       [0.24172427, 0.25029123, 0.3587265 , ..., 0.12683639, 0.02756822,
        0.12198409]], dtype=float32)

ValueError: Per-column arrays must each be 1-dimensional

In [31]:
from data_preparation import *

In [34]:
vectorizer = MatrixVectorizer()

all_vectorized_arr = np.concatenate([vectorizer.vectorize(matrix.cpu().numpy()) for matrix in pred_tensor])

In [36]:
all_vectorized_arr.shape

(4007136,)

In [37]:
df = pd.DataFrame({'ID': list(range(1, len(all_vectorized_arr)+1)), 'Predicted': all_vectorized_arr})

In [38]:
df

,ID,Predicted
0,1,0.168499
1,2,0.174591
2,3,0.503045
3,4,0.327084
4,5,0.176223
...,...,...
4007131,4007132,0.136374
4007132,4007133,0.062742
4007133,4007134,0.126836
4007134,4007135,0.027568
